# Refinitiv Data Library for Python
## Content - Pricing - Synchronous calls

This notebook demonstrates how to use the Pricing interface to retrieve data from the Refinitiv Data Platform.

## Import the library and load credentials

Credentials used by this and the other tutorials notebooks are stored in the **Configuration/credentials.ipynb** file.     

You should have edited the **Configuration/credentials.ipynb** to set your credentials as part of the **Quick Start** step.

In [1]:
import refinitiv.data as rd
from pandas import DataFrame
from IPython.display import display, clear_output

%run ../../Configuration/credentials.ipynb

## Open the session of your choice

Use our helper function in the Credentials notebook, **open_session(session_type)**, to create and open a session to connect to the 
- Refinitiv Data Platform directly (session_type="rdp") or via 
- Eikon 4 or Refinitiv Workspace (session_type="desktop") or via a 
- local realtime infrastructure (session_type="deployed").

You can also set a default in the **credentials** notebook

In [2]:
open_session('rdp')

<refinitiv.data.session._platform_session.Definition object at 0x15641757100 {session_name='session.platform.1'}>

## Get Price Snapshot
The following gets a snapshot from the non-streaming RDP Pricing API - note the **get_data()** call

In [3]:
# Define our RDP Snapshot Price object
response = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_data()

if (response.is_success):
    display(response.data.df)
else:
    print(response.http_status)

,Instruments,BID,ASK
0,EUR=,1.1846,1.1847
1,GBP=,1.3923,1.3924
2,JPY=,109.43,109.46
3,CAD=,1.2516,1.2517


## Alternative method for Snapshot data
If the above code generates an error, this could be because you are not licensed for the RDP Snapshot API OR you are using a **Deployed** session.   
If this is the case, then the following can be used to request a Snapshot from the Streaming data feed - note the **get_stream()** call

In [4]:
# Define our Streaming Price object
non_streaming = rd.content.pricing.Definition(
    ['EUR=', 'GBP=', 'JPY=', 'CAD='],
    fields=['BID', 'ASK']
).get_stream()

# We want to just snap the current prices, don't need updates
# Open the instrument in non-streaming mode
non_streaming.open(with_updates=False)
# Snapshot the prices at the time of the open() call
non_streaming.get_snapshot()

,Instrument,BID,ASK
0,EUR=,1.1845,1.1848
1,GBP=,1.3922,1.3925
2,JPY=,109.44,109.45
3,CAD=,1.2518,1.2519


The non-streaming requests are closed by the server - **no need to close them**.   

### Alternative ways of accessing instruments + values

#### Direct Access to fields

In [5]:
non_streaming['EUR=']['BID']

1.1845

In [6]:
eur = non_streaming['EUR=']
eur['ASK']

1.1848

#### Iterate on fields

In [7]:
print('JPY=')
for field_name, field_value in non_streaming['JPY=']:
    print(f"\t{field_name} : {field_value}")

JPY=
	BID : 109.44
	ASK : 109.45


#### Iterate on instruments and fields

In [8]:
for instrument in non_streaming:
    print(instrument.name)
    for field_name, field_value in instrument:
        print(f"\t{field_name} : {field_value}")

EUR=
	BID : 1.1845
	ASK : 1.1848
GBP=
	BID : 1.3922
	ASK : 1.3925
JPY=
	BID : 109.44
	ASK : 109.45
CAD=
	BID : 1.2518
	ASK : 1.2519


## Close the session

In [9]:
close_session()